In [1]:
import os
import time
import pandas as pd
from tqdm import notebook
from datetime import date, datetime

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
if not os.path.isdir('./html'):
    os.makedirs('./html')

In [3]:
df_items = None

for shop_name in ["munkonggadget","mercular.official.store"]:
    for p in notebook.tqdm(range(10)):
        url = f'https://shopee.co.th/{shop_name}?page={p}&sortBy=pop'
        # open website
        options = webdriver.ChromeOptions()
        options.add_argument(r"--user-data-dir=/Users/saratorn.s/Library/Application Support/Google/Chrome/Profile 1") 
#         options.add_argument("--headless=new")
        try:
            driver.quit()
        except Exception:
            pass
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        driver.execute_script("document.body.style.zoom='10%'")
        # scroll down
        for i in range(10):
            try:
                element = driver.find_element(By.XPATH, "/html/body")
                element.send_keys(Keys.PAGE_DOWN)
                break
            except Exception:
                time.sleep(2)
                pass
        # soup items
        while True:
            soup = BeautifulSoup(driver.page_source, 'lxml')
            item_class = "shop-search-result-view__item col-xs-2-4"
            items = soup.find_all('div', class_=item_class)
            time.sleep(5)
            if len(items) > 0:
                break
        # get the data
        for item in items:
            item_name = item.find('div', class_="NxFDlV").text.strip()
            price = item.find('span', class_="_0ZJOIv").text.strip()
            items_sold = item.find('div', class_="sPnnFI AGQcTN")
            if items_sold is None:
                items_sold = "ขายแล้ว 0 ชิ้น"
            else:
                items_sold = items_sold.text.strip()
            df_item = pd.DataFrame({'item_name':[item_name],
                                     'price':[price],
                                     'items_sold':[items_sold]})
            df_item['page'] = p + 1
            df_item['shop_name'] = shop_name
            df_items = pd.concat([df_items, df_item])

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
if not os.path.isdir('./data'):
    os.makedirs('./data')

In [ ]:
df_items['scrapped_date'] = date.today()
df_items.to_parquet(f'./data/gadget_shopee_{date.today()}.parquet')